In [1]:
"""
IR Spectroscopy Functional Group Classification Pipeline
Uses SMILES strings to automatically label functional groups
Processes real IR spectra data for multi-label classification
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
from tqdm import tqdm

# ML Libraries
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import (
    classification_report, accuracy_score, f1_score, 
    hamming_loss, jaccard_score
)
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

# Chemistry Libraries
try:
    from rdkit import Chem
    from rdkit.Chem import Descriptors, rdMolDescriptors
    print("✓ RDKit loaded successfully")
except ImportError:
    print("Installing RDKit...")
    import subprocess
    subprocess.check_call(['pip', 'install', 'rdkit'])
    from rdkit import Chem
    from rdkit.Chem import Descriptors, rdMolDescriptors
    print("✓ RDKit installed and loaded")

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

print("=" * 80)
print("IR SPECTROSCOPY FUNCTIONAL GROUP CLASSIFICATION")
print("Real Data Pipeline with SMILES-based Labeling")
print("=" * 80)


✓ RDKit loaded successfully
IR SPECTROSCOPY FUNCTIONAL GROUP CLASSIFICATION
Real Data Pipeline with SMILES-based Labeling


In [2]:

# ============================================================================
# PART 1: FUNCTIONAL GROUP DETECTION FROM SMILES
# ============================================================================

# Define 31 functional groups with SMARTS patterns
FUNCTIONAL_GROUP_PATTERNS = {
    'O-H_alcohol_free': '[OX2H]',
    'O-H_alcohol_bonded': '[OH]',
    'N-H_primary_amine': '[NX3;H2]',
    'N-H_secondary_amine': '[NX3;H1]',
    'O-H_carboxylic_acid': '[CX3](=O)[OX2H1]',
    'C-H_alkyne': '[CX2]#[CX2]',
    'C-H_aromatic': 'c',
    'C-H_alkene': '[CX3]=[CX3]',
    'C-H_alkane': '[CX4]',
    'C-H_aldehyde': '[CX3H1](=O)',
    'CN_nitrile': '[CX2]#[NX1]',
    'CC_alkyne': '[CX2]#[CX2]',
    'NCO_isocyanate': '[NX2]=C=O',
    'CO_acid_chloride': '[CX3](=[OX1])[Cl]',
    'CO_anhydride': '[CX3](=[OX1])[OX2][CX3](=[OX1])',
    'CO_ester': '[CX3](=O)[OX2H0]',
    'CO_aldehyde': '[CX3H1](=O)[#6]',
    'CO_ketone': '[CX3](=O)[#6]',
    'CO_carboxylic_acid': '[CX3](=O)[OX2H1]',
    'CO_amide': '[CX3](=[OX1])[NX3]',
    'CC_aromatic': 'c:c',
    'CC_alkene': '[CX3]=[CX3]',
    'CN_imine': '[CX3]=[NX2]',
    'NH_amide_bend': '[NX3][CX3](=[OX1])',
    'NH2_amine_bend': '[NX3;H2]',
    'CO_ester_stretch': '[CX3](=O)[OX2]',
    'CO_alcohol': '[CX4][OX2H]',
    'CO_ether': '[OX2]([#6])[#6]',
    'CN_amine': '[CX4][NX3]',
    'SO_sulfone': '[SX4](=O)(=O)',
    'CX_halide': '[#6][F,Cl,Br,I]'
}

def detect_functional_groups(smiles):
    """
    Detect functional groups from SMILES string using SMARTS patterns
    
    Parameters:
    -----------
    smiles : str
        SMILES string of molecule
        
    Returns:
    --------
    dict : Binary labels for each functional group
    """
    mol = Chem.MolFromSmiles(smiles)
    
    if mol is None:
        # Invalid SMILES - return all zeros
        return {group: 0 for group in FUNCTIONAL_GROUP_PATTERNS.keys()}
    
    labels = {}
    for group_name, smarts_pattern in FUNCTIONAL_GROUP_PATTERNS.items():
        pattern = Chem.MolFromSmarts(smarts_pattern)
        if pattern is not None:
            matches = mol.GetSubstructMatches(pattern)
            labels[group_name] = 1 if len(matches) > 0 else 0
        else:
            labels[group_name] = 0
    
    return labels

print("\n[STEP 1] Loading SMILES and Generating Labels...")
print("-" * 80)

# Load SMILES file
SMILES_FILE = 'Dataset/smiles.txt'
smiles_list = []

with open(SMILES_FILE, 'r') as f:
    for line in f:
        smiles_list.append(line.strip())

print(f"✓ Loaded {len(smiles_list)} SMILES strings")

# Generate functional group labels (process in batches for large datasets)
print("\nDetecting functional groups from SMILES...")
all_labels = []
batch_size = 1000

for i in tqdm(range(0, len(smiles_list), batch_size)):
    batch = smiles_list[i:i+batch_size]
    batch_labels = [detect_functional_groups(smi) for smi in batch]
    all_labels.extend(batch_labels)

# Create labels dataframe
labels_df = pd.DataFrame(all_labels)
print(f"\n✓ Generated labels for {len(labels_df)} compounds")
print(f"✓ Number of functional groups: {labels_df.shape[1]}")

# Show distribution
print("\nFunctional Group Distribution:")
fg_counts = labels_df.sum().sort_values(ascending=False)
print(fg_counts.head(15))










[STEP 1] Loading SMILES and Generating Labels...
--------------------------------------------------------------------------------
✓ Loaded 127468 SMILES strings

Detecting functional groups from SMILES...


100%|██████████| 128/128 [01:15<00:00,  1.70it/s]




✓ Generated labels for 127468 compounds
✓ Number of functional groups: 31

Functional Group Distribution:
C-H_alkane             121823
CO_ether                56453
O-H_alcohol_bonded      42347
O-H_alcohol_free        42347
CN_amine                40848
CO_ketone               38395
CO_alcohol              37486
N-H_secondary_amine     23071
C-H_aromatic            21509
CC_aromatic             18805
C-H_aldehyde            17706
CC_alkene               16234
C-H_alkene              16234
CN_nitrile              15776
CC_alkyne               15623
dtype: int64


In [ ]:
# ============================================================================
# PART 2A: LOAD IR SPECTRA DATA (PHASE 1 - Read in Chunks)
# ============================================================================

print("\n[STEP 2A] Loading IR Spectra Data (Phase 1: Reading chunks)...")
print("-" * 80)

IR_DATA_FILE = 'Dataset/qm9s_irdata.csv'

# Configuration
MAX_SAMPLES = 100000  # Adjust based on available memory
chunk_size = 1000

print(f"Target: Loading first {MAX_SAMPLES} samples")
print(f"Chunk size: {chunk_size} rows per chunk")
print("Reading IR spectra (this may take a few minutes)...\n")

# Read data in chunks
chunks = []
total_loaded = 0

for chunk in pd.read_csv(IR_DATA_FILE, chunksize=chunk_size, header=None, low_memory=False):
    chunks.append(chunk)
    total_loaded += len(chunk)
    
    if len(chunks) % 10 == 0:
        print(f"  Processed {total_loaded} spectra...")
    
    # Stop loading if we've reached the limit
    if total_loaded >= MAX_SAMPLES:
        print(f"  Reached {MAX_SAMPLES} sample limit")
        break

print(f"\n✓ Loaded {len(chunks)} chunks containing {total_loaded} total spectra")
print(f"✓ Memory: Chunks stored as list (next cell will combine them)")
print("\n⚠️ Run next cell to combine chunks into DataFrame")


[STEP 2A] Loading IR Spectra Data (Phase 1: Reading chunks)...
--------------------------------------------------------------------------------
Target: Loading first 50000 samples
Chunk size: 1000 rows per chunk
Reading IR spectra (this may take a few minutes)...

  Processed 10000 spectra...
  Processed 10000 spectra...
  Processed 20000 spectra...
  Processed 20000 spectra...
  Processed 30000 spectra...
  Processed 30000 spectra...
  Processed 40000 spectra...
  Processed 40000 spectra...
  Processed 50000 spectra...
  Reached 50000 sample limit

✓ Loaded 50 chunks containing 50000 total spectra
✓ Memory: Chunks stored as list (next cell will combine them)

⚠️ Run next cell to combine chunks into DataFrame
  Processed 50000 spectra...
  Reached 50000 sample limit

✓ Loaded 50 chunks containing 50000 total spectra
✓ Memory: Chunks stored as list (next cell will combine them)

⚠️ Run next cell to combine chunks into DataFrame


In [7]:
# ============================================================================
# PART 2B: COMBINE CHUNKS (PHASE 2 - Concatenate Data)
# ============================================================================

print("\n[STEP 2B] Combining Data Chunks (Phase 2)...")
print("-" * 80)

# Check if chunks variable exists and is valid
try:
    # This will raise NameError if chunks doesn't exist
    len(chunks)
    if len(chunks) == 0:
        print("❌ ERROR: 'chunks' list is empty!")
        print("\nThe previous cell (PART 2A) may have failed or not loaded any data.")
        print("Please re-run the chunk loading cell (PART 2A) and ensure it completes successfully.")
        raise ValueError("chunks list is empty")
except NameError:
    print("❌ ERROR: 'chunks' variable not found!")
    print("\nThis means you haven't run the previous cell (PART 2A) yet.")
    print("Please go back and run the chunk loading cell (PART 2A) first, then run this cell.")
    print("\nProper order:")
    print("  1. Run cell 1 (imports)")
    print("  2. Run cell 2 (SMILES processing)")  
    print("  3. Run cell 3 (PART 2A - load chunks)")
    print("  4. Run cell 4 (PART 2B - combine chunks) ← YOU ARE HERE")
    raise

print(f"Combining {len(chunks)} chunks into single DataFrame...")

# Concatenate in smaller batches to reduce memory spikes
if len(chunks) > 20:
    print("Using batch concatenation to reduce memory pressure...")
    batch_size = 20
    combined_chunks = []
    
    for i in range(0, len(chunks), batch_size):
        batch = chunks[i:i+batch_size]
        combined_chunks.append(pd.concat(batch, ignore_index=True))
        chunks[i:i+batch_size] = []  # Free memory immediately
        
        if (i // batch_size) % 5 == 0:
            print(f"  Combined batch {i//batch_size + 1}/{(len(chunks)-1)//batch_size + 1}")
    
    print("\nFinal concatenation of batches...")
    ir_data = pd.concat(combined_chunks, ignore_index=True)
    del combined_chunks  # Free memory
else:
    ir_data = pd.concat(chunks, ignore_index=True)

# Clear chunks from memory
chunks = []
import gc
gc.collect()

print(f"\n✓ IR data combined: {ir_data.shape}")
print(f"✓ Memory: Chunks cleared, DataFrame ready")
print("\n⚠️ Run next cell to clean and align data")


[STEP 2B] Combining Data Chunks (Phase 2)...
--------------------------------------------------------------------------------
Combining 20 chunks into single DataFrame...

✓ IR data combined: (20000, 3001)
✓ Memory: Chunks cleared, DataFrame ready

⚠️ Run next cell to clean and align data

✓ IR data combined: (20000, 3001)
✓ Memory: Chunks cleared, DataFrame ready

⚠️ Run next cell to clean and align data


In [8]:
# ============================================================================
# PART 2C: CLEAN DATA (PHASE 3 - Remove Non-Numeric Columns)
# ============================================================================

print("\n[STEP 2C] Cleaning IR Data (Phase 3)...")
print("-" * 80)

print("Identifying and removing non-numeric columns...")
numeric_cols = []

for col in ir_data.columns:
    try:
        # Try to convert column to float
        pd.to_numeric(ir_data[col], errors='raise')
        numeric_cols.append(col)
    except (ValueError, TypeError):
        # Skip non-numeric columns
        print(f"  Skipping non-numeric column: {col}")
        continue

print(f"\n✓ Found {len(numeric_cols)} numeric columns out of {len(ir_data.columns)} total")

# Keep only numeric columns
ir_data = ir_data[numeric_cols]

print(f"✓ Cleaned IR data shape: {ir_data.shape}")
print("\n⚠️ Run next cell to align with SMILES labels")


[STEP 2C] Cleaning IR Data (Phase 3)...
--------------------------------------------------------------------------------
Identifying and removing non-numeric columns...
  Skipping non-numeric column: 0

✓ Found 3000 numeric columns out of 3001 total

✓ Found 3000 numeric columns out of 3001 total
✓ Cleaned IR data shape: (20000, 3000)

⚠️ Run next cell to align with SMILES labels
✓ Cleaned IR data shape: (20000, 3000)

⚠️ Run next cell to align with SMILES labels


In [9]:
# ============================================================================
# PART 2D: ALIGN DATA (PHASE 4 - Match IR Data with SMILES Labels)
# ============================================================================

print("\n[STEP 2D] Aligning IR Data with SMILES Labels (Phase 4)...")
print("-" * 80)

print(f"SMILES labels available: {len(smiles_list)} samples")
print(f"IR spectra available: {len(ir_data)} samples")

# Ensure alignment
n_samples = min(len(smiles_list), len(ir_data))

print(f"\n✓ Using first {n_samples} samples (minimum of both datasets)")

# Trim both datasets to same size
ir_data = ir_data.iloc[:n_samples]
labels_df = labels_df.iloc[:n_samples]

print(f"✓ Final aligned dataset: {n_samples} samples")
print(f"✓ IR data shape: {ir_data.shape}")
print(f"✓ Labels shape: {labels_df.shape}")

# Free up memory
import gc
gc.collect()

print("\n" + "=" * 80)
print("DATA LOADING COMPLETE!")
print("=" * 80)
print("✓ All phases finished successfully")
print("✓ Data is ready for feature extraction")
print("\n⚠️ You can now proceed to the next cell for feature extraction")


[STEP 2D] Aligning IR Data with SMILES Labels (Phase 4)...
--------------------------------------------------------------------------------
SMILES labels available: 127468 samples
IR spectra available: 20000 samples

✓ Using first 20000 samples (minimum of both datasets)
✓ Final aligned dataset: 20000 samples
✓ IR data shape: (20000, 3000)
✓ Labels shape: (20000, 31)

DATA LOADING COMPLETE!
✓ All phases finished successfully
✓ Data is ready for feature extraction

⚠️ You can now proceed to the next cell for feature extraction


# 📊 Enhanced Feature Engineering Pipeline

This section includes comprehensive preprocessing and feature extraction:

## 🔍 Part 3A: Exploratory Data Analysis (EDA)
- Dataset overview and statistics
- Raw spectra visualization (4 random samples)
- Intensity distribution analysis
- Data quality checks

## 🛠️ Part 3B: Preprocessing Pipeline
1. **Baseline Correction** - Asymmetric Least Squares (ALS) method
   - Removes baseline drift and background signals
   - Parameters: λ=1e5 (smoothness), p=0.01 (asymmetry)

2. **Spectral Smoothing** - Savitzky-Golay filter
   - Reduces noise while preserving peak shapes
   - Parameters: window=11, polynomial order=3

3. **Normalization** - Min-Max scaling
   - Standardizes intensity ranges across samples
   - Enables fair comparison between spectra

## 🎯 Part 3C: Advanced Feature Extraction
Extracts **~196 features** per spectrum:
- **186 regional features**: 31 regions × 6 statistics (max, mean, std, sum, median, variance)
- **4 peak features**: peak count, average/max height, height std
- **4 derivative features**: 1st and 2nd derivative statistics
- **2 spectral moments**: weighted position information

## 📈 Part 3D: Feature Analysis
- Correlation heatmap (identifies redundant features)
- Variance analysis (removes uninformative features)
- Feature distribution visualization
- Label balance analysis

**Result**: Clean, normalized, feature-rich dataset ready for ML models!

In [ ]:
# ============================================================================
# PART 3A: EXPLORATORY DATA ANALYSIS (EDA) - Raw Spectra
# ============================================================================

print("\n[STEP 3A] Exploratory Data Analysis - Raw IR Spectra...")
print("=" * 80)

# Basic statistics
print("\n1. Dataset Overview:")
print(f"   - Number of samples: {len(ir_data)}")
print(f"   - Number of spectral points per sample: {ir_data.shape[1]}")
print(f"   - Memory usage: {ir_data.memory_usage(deep=True).sum() / (1024**2):.2f} MB")

# Check for missing values
missing_values = ir_data.isnull().sum().sum()
print(f"\n2. Data Quality:")
print(f"   - Missing values: {missing_values}")
print(f"   - Data type: {ir_data.dtypes[0]}")

# Statistical summary
print("\n3. Statistical Summary of Spectral Intensities:")
spectral_stats = ir_data.describe().T
print(spectral_stats[['mean', 'std', 'min', 'max']].head(10))

# Visualize sample spectra
print("\n4. Visualizing Sample Spectra...")
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle('Raw IR Spectra - Sample Visualization', fontsize=16, fontweight='bold')

# Plot 4 random samples
sample_indices = np.random.choice(len(ir_data), size=4, replace=False)

for idx, ax in enumerate(axes.flat):
    sample_idx = sample_indices[idx]
    spectrum = ir_data.iloc[sample_idx].values
    wavenumber = np.arange(len(spectrum))
    
    ax.plot(wavenumber, spectrum, linewidth=0.8, color='darkblue', alpha=0.7)
    ax.set_title(f'Sample {sample_idx}', fontsize=12, fontweight='bold')
    ax.set_xlabel('Spectral Point Index', fontsize=10)
    ax.set_ylabel('Intensity', fontsize=10)
    ax.grid(True, alpha=0.3)
    ax.set_xlim(0, len(spectrum))

plt.tight_layout()
plt.show()

# Intensity distribution
print("\n5. Analyzing Intensity Distribution...")
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Overall intensity distribution
all_intensities = ir_data.values.flatten()
axes[0].hist(all_intensities, bins=100, color='steelblue', edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Intensity', fontsize=11, fontweight='bold')
axes[0].set_ylabel('Frequency', fontsize=11, fontweight='bold')
axes[0].set_title('Distribution of All Spectral Intensities', fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3, axis='y')

# Box plot of intensities per sample (subset)
sample_subset = ir_data.iloc[:100].T  # First 100 samples
axes[1].boxplot([sample_subset[col].values for col in sample_subset.columns[:20]], 
                showfliers=False)
axes[1].set_xlabel('Sample Index (first 20 shown)', fontsize=11, fontweight='bold')
axes[1].set_ylabel('Intensity', fontsize=11, fontweight='bold')
axes[1].set_title('Intensity Range Across Samples', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\n✓ EDA complete - proceeding to preprocessing...")





In [ ]:
# ============================================================================
# PART 3B: DATA PREPROCESSING - Baseline Correction & Normalization
# ============================================================================

print("\n[STEP 3B] Preprocessing IR Spectra...")
print("=" * 80)

from scipy import signal
from scipy.ndimage import uniform_filter1d

def baseline_correction_als(spectrum, lam=1e6, p=0.01, niter=10):
    """
    Asymmetric Least Squares baseline correction
    
    Parameters:
    -----------
    spectrum : array
        Raw spectrum
    lam : float
        Smoothness parameter (larger = smoother baseline)
    p : float
        Asymmetry parameter (0 < p < 1, smaller = more asymmetric)
    niter : int
        Number of iterations
    """
    L = len(spectrum)
    D = np.diff(np.eye(L), 2)
    w = np.ones(L)
    
    for i in range(niter):
        W = np.diag(w)
        Z = W + lam * D.T @ D
        z = np.linalg.solve(Z, w * spectrum)
        w = p * (spectrum > z) + (1 - p) * (spectrum < z)
    
    return spectrum - z

def savitzky_golay_smooth(spectrum, window_length=11, polyorder=3):
    """
    Savitzky-Golay smoothing filter
    
    Parameters:
    -----------
    spectrum : array
        Spectrum to smooth
    window_length : int
        Window size (must be odd)
    polyorder : int
        Polynomial order
    """
    if window_length % 2 == 0:
        window_length += 1
    return signal.savgol_filter(spectrum, window_length, polyorder)

def normalize_spectrum(spectrum, method='minmax'):
    """
    Normalize spectrum
    
    Parameters:
    -----------
    spectrum : array
        Spectrum to normalize
    method : str
        'minmax' or 'standard'
    """
    if method == 'minmax':
        min_val = np.min(spectrum)
        max_val = np.max(spectrum)
        if max_val - min_val > 0:
            return (spectrum - min_val) / (max_val - min_val)
        return spectrum
    elif method == 'standard':
        mean_val = np.mean(spectrum)
        std_val = np.std(spectrum)
        if std_val > 0:
            return (spectrum - mean_val) / std_val
        return spectrum
    return spectrum

# Apply preprocessing
print("\n1. Applying preprocessing pipeline...")
print("   - Baseline correction (ALS)")
print("   - Spectral smoothing (Savitzky-Golay)")
print("   - Normalization (Min-Max)")

preprocessed_spectra = []

for idx in tqdm(range(len(ir_data)), desc="Preprocessing spectra"):
    spectrum = ir_data.iloc[idx].values
    
    # Step 1: Baseline correction
    spectrum_corrected = baseline_correction_als(spectrum, lam=1e5, p=0.01)
    
    # Step 2: Smoothing
    spectrum_smooth = savitzky_golay_smooth(spectrum_corrected, window_length=11, polyorder=3)
    
    # Step 3: Normalization
    spectrum_normalized = normalize_spectrum(spectrum_smooth, method='minmax')
    
    preprocessed_spectra.append(spectrum_normalized)

# Create preprocessed DataFrame
ir_data_preprocessed = pd.DataFrame(preprocessed_spectra)

print(f"\n✓ Preprocessing complete")
print(f"✓ Preprocessed data shape: {ir_data_preprocessed.shape}")

# Visualize before/after preprocessing
print("\n2. Visualizing Preprocessing Effects...")
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle('Preprocessing Effect on IR Spectra', fontsize=16, fontweight='bold')

sample_indices = np.random.choice(len(ir_data), size=2, replace=False)

for i, sample_idx in enumerate(sample_indices):
    # Raw spectrum
    raw_spectrum = ir_data.iloc[sample_idx].values
    preprocessed_spectrum = ir_data_preprocessed.iloc[sample_idx].values
    x_axis = np.arange(len(raw_spectrum))
    
    # Before preprocessing
    axes[i, 0].plot(x_axis, raw_spectrum, linewidth=0.8, color='darkred', alpha=0.8)
    axes[i, 0].set_title(f'Sample {sample_idx} - Raw Spectrum', fontsize=12, fontweight='bold')
    axes[i, 0].set_xlabel('Spectral Point', fontsize=10)
    axes[i, 0].set_ylabel('Intensity', fontsize=10)
    axes[i, 0].grid(True, alpha=0.3)
    
    # After preprocessing
    axes[i, 1].plot(x_axis, preprocessed_spectrum, linewidth=0.8, color='darkgreen', alpha=0.8)
    axes[i, 1].set_title(f'Sample {sample_idx} - Preprocessed Spectrum', fontsize=12, fontweight='bold')
    axes[i, 1].set_xlabel('Spectral Point', fontsize=10)
    axes[i, 1].set_ylabel('Normalized Intensity', fontsize=10)
    axes[i, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✓ Preprocessing visualization complete")

In [ ]:
# ============================================================================
# PART 3C: ADVANCED FEATURE EXTRACTION
# ============================================================================

print("\n[STEP 3C] Advanced Feature Extraction from Preprocessed Spectra...")
print("=" * 80)

def extract_advanced_spectral_features(spectrum_row):
    """
    Extract comprehensive statistical and spectral features
    
    Features extracted:
    - Regional statistics (max, mean, std, sum, median, variance)
    - Peak features (number of peaks, peak heights, peak positions)
    - Derivative features (1st and 2nd derivatives)
    - Spectral moments
    """
    spectrum = spectrum_row.values
    
    # Divide spectrum into regions
    n_regions = 31
    region_size = len(spectrum) // n_regions
    
    features = []
    
    # Regional statistical features
    for i in range(n_regions):
        start = i * region_size
        end = (i + 1) * region_size if i < n_regions - 1 else len(spectrum)
        region = spectrum[start:end]
        
        if len(region) > 0:
            features.extend([
                np.max(region),          # Maximum intensity
                np.mean(region),         # Mean intensity
                np.std(region),          # Standard deviation
                np.sum(region),          # Integrated area
                np.median(region),       # Median intensity
                np.var(region),          # Variance
            ])
        else:
            features.extend([0, 0, 0, 0, 0, 0])
    
    # Peak detection features
    peaks, properties = signal.find_peaks(spectrum, height=0.1, distance=5)
    
    # Peak statistics
    features.extend([
        len(peaks),                                    # Number of peaks
        np.mean(properties['peak_heights']) if len(peaks) > 0 else 0,  # Average peak height
        np.max(properties['peak_heights']) if len(peaks) > 0 else 0,   # Max peak height
        np.std(properties['peak_heights']) if len(peaks) > 0 else 0,   # Peak height std
    ])
    
    # Derivative features (for capturing spectral shape changes)
    first_derivative = np.gradient(spectrum)
    second_derivative = np.gradient(first_derivative)
    
    features.extend([
        np.mean(np.abs(first_derivative)),   # Mean absolute 1st derivative
        np.std(first_derivative),             # Std of 1st derivative
        np.mean(np.abs(second_derivative)),   # Mean absolute 2nd derivative
        np.std(second_derivative),            # Std of 2nd derivative
    ])
    
    # Spectral moments
    moments = [
        np.sum(spectrum * np.arange(len(spectrum))),  # 1st moment
        np.sum(spectrum * np.arange(len(spectrum))**2), # 2nd moment
    ]
    features.extend(moments)
    
    return features

# Extract features
print("\n1. Extracting features from preprocessed spectra...")
print("   This includes:")
print("   - 186 regional features (31 regions × 6 statistics)")
print("   - 4 peak-based features")
print("   - 4 derivative features")
print("   - 2 spectral moment features")
print("   Total: ~196 features per spectrum\n")

X_features = []

for idx in tqdm(range(len(ir_data_preprocessed)), desc="Feature extraction"):
    features = extract_advanced_spectral_features(ir_data_preprocessed.iloc[idx])
    X_features.append(features)

X = pd.DataFrame(X_features)
y = labels_df

print(f"\n✓ Feature extraction complete")
print(f"✓ Feature matrix shape: {X.shape}")
print(f"✓ Label matrix shape: {y.shape}")

# Feature statistics
print("\n2. Feature Statistics:")
print(X.describe().T[['mean', 'std', 'min', 'max']].head(10))

# Check for any NaN or infinite values
nan_count = X.isnull().sum().sum()
inf_count = np.isinf(X.values).sum()

print(f"\n3. Data Quality Check:")
print(f"   - NaN values: {nan_count}")
print(f"   - Infinite values: {inf_count}")

if nan_count > 0 or inf_count > 0:
    print("   - Cleaning: Replacing NaN/Inf with 0...")
    X = X.replace([np.inf, -np.inf], np.nan).fillna(0)
    print("   ✓ Cleaned")

print("\n✓ Advanced feature extraction complete")

In [ ]:
# ============================================================================
# PART 3D: FEATURE ANALYSIS & VISUALIZATION
# ============================================================================

print("\n[STEP 3D] Feature Analysis & Correlation Study...")
print("=" * 80)

# Feature correlation analysis
print("\n1. Computing feature correlation matrix...")
feature_correlation = X.corr()

# Visualize correlation heatmap (subset for readability)
print("\n2. Visualizing feature correlations (first 50 features)...")
fig, ax = plt.subplots(figsize=(14, 12))
sns.heatmap(feature_correlation.iloc[:50, :50], 
            cmap='coolwarm', center=0, 
            square=True, linewidths=0.5,
            cbar_kws={"shrink": 0.8},
            ax=ax)
ax.set_title('Feature Correlation Matrix (First 50 Features)', 
             fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

# Identify highly correlated features
print("\n3. Identifying highly correlated feature pairs...")
correlation_threshold = 0.95
high_corr_pairs = []

for i in range(len(feature_correlation.columns)):
    for j in range(i+1, len(feature_correlation.columns)):
        if abs(feature_correlation.iloc[i, j]) > correlation_threshold:
            high_corr_pairs.append({
                'feature_1': i,
                'feature_2': j,
                'correlation': feature_correlation.iloc[i, j]
            })

print(f"   Found {len(high_corr_pairs)} highly correlated pairs (|r| > {correlation_threshold})")
if len(high_corr_pairs) > 0:
    print(f"   First 5 pairs:")
    for pair in high_corr_pairs[:5]:
        print(f"   - Features {pair['feature_1']} & {pair['feature_2']}: r = {pair['correlation']:.3f}")

# Feature variance analysis
print("\n4. Analyzing feature variance...")
feature_variance = X.var()
low_variance_features = feature_variance[feature_variance < 0.001].index.tolist()

print(f"   Features with very low variance (< 0.001): {len(low_variance_features)}")
if len(low_variance_features) > 0:
    print(f"   These features may not be informative for classification")

# Visualize feature distributions
print("\n5. Visualizing sample feature distributions...")
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
fig.suptitle('Distribution of Selected Features', fontsize=16, fontweight='bold')

# Select 6 features with good variance
selected_features = feature_variance.nlargest(6).index

for idx, (ax, feature_idx) in enumerate(zip(axes.flat, selected_features)):
    ax.hist(X[feature_idx], bins=50, color='teal', edgecolor='black', alpha=0.7)
    ax.set_title(f'Feature {feature_idx}', fontsize=11, fontweight='bold')
    ax.set_xlabel('Value', fontsize=10)
    ax.set_ylabel('Frequency', fontsize=10)
    ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# Label distribution analysis
print("\n6. Functional Group Label Distribution:")
label_counts = y.sum().sort_values(ascending=False)
print(label_counts.head(15))

# Visualize label distribution
fig, ax = plt.subplots(figsize=(16, 6))
label_counts.plot(kind='bar', ax=ax, color='steelblue', edgecolor='black')
ax.set_title('Functional Group Distribution Across Dataset', 
             fontsize=14, fontweight='bold')
ax.set_xlabel('Functional Group', fontsize=11, fontweight='bold')
ax.set_ylabel('Number of Samples', fontsize=11, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

print("\n✓ Feature analysis complete")

In [11]:
# ============================================================================
# PART 4: TRAIN-TEST SPLIT AND SCALING
# ============================================================================

print("\n[STEP 4] Preparing Data for Training...")
print("-" * 80)

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"✓ Training set: {X_train.shape[0]} samples ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"✓ Test set: {X_test.shape[0]} samples ({X_test.shape[0]/len(X)*100:.1f}%)")

# Identify usable labels: columns that contain at least two classes in the training set
print("\nAnalyzing label distribution...")
train_label_uniques = y_train.nunique()
valid_label_cols = train_label_uniques[train_label_uniques == 2].index.tolist()
invalid_label_cols = [col for col in y_train.columns if col not in valid_label_cols]

if len(valid_label_cols) == 0:
    print("\n⚠️ WARNING: No label columns have both classes in the training set!")
    print("This typically happens when:")
    print("  1. MAX_SAMPLES is too small (increase it in cell 3)")
    print("  2. Your SMILES data doesn't have diverse functional groups")
    print("  3. Data alignment issue between SMILES and IR spectra")
    print("\nAttempting to use all available labels anyway...")
    
    # Use all columns that have at least some variation
    valid_label_cols = y_train.columns.tolist()
    
    if len(valid_label_cols) == 0:
        raise ValueError(
            "No usable labels found. Please check your data alignment and increase MAX_SAMPLES."
        )

if len(invalid_label_cols) > 0:
    print(f"\n✓ Found {len(valid_label_cols)} valid label columns")
    print(f"⚠️ Skipping {len(invalid_label_cols)} labels with single class in training set")
    # Show a small subset to avoid flooding the console
    if len(invalid_label_cols) <= 5:
        print(f"  Skipped labels: {invalid_label_cols}")
    else:
        preview = invalid_label_cols[:5]
        print(f"  First 5 skipped: {preview} ...")
else:
    print(f"\n✓ All {len(valid_label_cols)} labels are valid for training")

# Filter labels to only valid columns downstream
y_train = y_train[valid_label_cols]
y_test = y_test[valid_label_cols]

print(f"✓ Using {len(valid_label_cols)} functional groups for classification")

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("✓ Features scaled using StandardScaler")



[STEP 4] Preparing Data for Training...
--------------------------------------------------------------------------------
✓ Training set: 16000 samples (80.0%)
✓ Test set: 4000 samples (20.0%)

Analyzing label distribution...

✓ Found 25 valid label columns
⚠️ Skipping 6 labels with single class in training set
  First 5 skipped: ['O-H_carboxylic_acid', 'NCO_isocyanate', 'CO_acid_chloride', 'CO_anhydride', 'CO_carboxylic_acid'] ...
✓ Using 25 functional groups for classification
✓ Features scaled using StandardScaler
✓ Using 25 functional groups for classification
✓ Features scaled using StandardScaler


In [12]:
# ============================================================================
# PART 5: TRAIN MACHINE LEARNING MODELS
# ============================================================================

print("\n[STEP 5] Training Machine Learning Models...")
print("=" * 80)

models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'Random Forest': RandomForestClassifier(
        n_estimators=100, max_depth=15, random_state=42, n_jobs=-1
    ),
    'Gradient Boosting': GradientBoostingClassifier(
        n_estimators=50, max_depth=5, random_state=42
    ),
    'Neural Network': MLPClassifier(
        hidden_layer_sizes=(128, 64), max_iter=300,
        random_state=42, early_stopping=True
    )
}

results = {}

for name, model in models.items():
    print(f"\nTraining {name}...")
    
    # Wrap for multi-label classification
    clf = MultiOutputClassifier(model, n_jobs=-1)
    
    # Train
    clf.fit(X_train_scaled, y_train)
    
    # Predict
    y_pred = clf.predict(X_test_scaled)
    
    # Calculate metrics
    results[name] = {
        'model': clf,
        'predictions': y_pred,
        'Accuracy': accuracy_score(y_test, y_pred),
        'F1-Micro': f1_score(y_test, y_pred, average='micro', zero_division=0),
        'F1-Macro': f1_score(y_test, y_pred, average='macro', zero_division=0),
        'Hamming Loss': hamming_loss(y_test, y_pred),
        'Jaccard Score': jaccard_score(y_test, y_pred, average='samples', zero_division=0)
    }
    
    print(f"  ✓ Accuracy: {results[name]['Accuracy']:.4f}")
    print(f"  ✓ F1-Micro: {results[name]['F1-Micro']:.4f}")
    print(f"  ✓ F1-Macro: {results[name]['F1-Macro']:.4f}")




[STEP 5] Training Machine Learning Models...

Training Logistic Regression...
  ✓ Accuracy: 0.0425
  ✓ F1-Micro: 0.3678
  ✓ F1-Macro: 0.0852

Training Random Forest...
  ✓ Accuracy: 0.0425
  ✓ F1-Micro: 0.3678
  ✓ F1-Macro: 0.0852

Training Random Forest...
  ✓ Accuracy: 0.0573
  ✓ F1-Micro: 0.4126
  ✓ F1-Macro: 0.1196

Training Gradient Boosting...
  ✓ Accuracy: 0.0573
  ✓ F1-Micro: 0.4126
  ✓ F1-Macro: 0.1196

Training Gradient Boosting...


: 

In [ ]:
# ============================================================================
# PART 6: MODEL COMPARISON AND EVALUATION
# ============================================================================

print("\n[STEP 6] Model Performance Comparison...")
print("=" * 80)

comparison_data = {
    name: {k: v for k, v in info.items() if k not in ['model', 'predictions']}
    for name, info in results.items()
}

comparison_df = pd.DataFrame(comparison_data).T
print("\nModel Performance:")
print(comparison_df.to_string())

# Find best model
best_model_name = comparison_df['F1-Micro'].idxmax()
print(f"\n{'='*80}")
print(f"BEST MODEL: {best_model_name}")
print(f"{'='*80}")
print(f"F1-Micro: {comparison_df.loc[best_model_name, 'F1-Micro']:.4f}")
print(f"Accuracy: {comparison_df.loc[best_model_name, 'Accuracy']:.4f}")

# Visualize comparison
fig, ax = plt.subplots(figsize=(12, 6))
comparison_df[['Accuracy', 'F1-Micro', 'F1-Macro', 'Jaccard Score']].plot(
    kind='bar', ax=ax, colormap='viridis'
)
ax.set_title('Model Performance Comparison', fontsize=14, fontweight='bold')
ax.set_ylabel('Score')
ax.legend(loc='lower right')
ax.grid(True, alpha=0.3, axis='y')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()



In [ ]:
# ============================================================================
# PART 7: DETAILED ANALYSIS
# ============================================================================

print("\n[STEP 7] Detailed Per-Functional-Group Analysis...")
print("-" * 80)

best_predictions = results[best_model_name]['predictions']

# Classification report
report = classification_report(
    y_test, best_predictions,
    target_names=list(y_test.columns),
    zero_division=0,
    output_dict=True
)

report_df = pd.DataFrame(report).T
print("\nTop 10 Functional Groups (by F1-Score):")
print(report_df.iloc[:-3][['precision', 'recall', 'f1-score', 'support']].head(10))

# Visualize functional group distribution
fig, ax = plt.subplots(figsize=(16, 6))
y_train.sum().sort_values(ascending=False).plot(kind='bar', ax=ax, color='steelblue')
ax.set_title('Functional Group Distribution in Training Data', fontsize=14, fontweight='bold')
ax.set_xlabel('Functional Group')
ax.set_ylabel('Number of Occurrences')
ax.grid(True, alpha=0.3, axis='y')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()



In [ ]:
# ============================================================================
# PART 8: SAVE RESULTS
# ============================================================================

print("\n[STEP 8] Saving Results...")
print("-" * 80)

# Save labels
labels_df.to_csv('functional_group_labels.csv', index=False)
print("✓ Saved: functional_group_labels.csv")

# Save model comparison
comparison_df.to_csv('model_comparison_results.csv')
print("✓ Saved: model_comparison_results.csv")

# Save detailed performance
report_df.to_csv('per_group_performance.csv')
print("✓ Saved: per_group_performance.csv")

# Save predictions
pred_df = pd.DataFrame(
    best_predictions, 
    columns=list(y_test.columns)
)
pred_df.to_csv('test_predictions.csv', index=False)
print("✓ Saved: test_predictions.csv")

print("\n" + "=" * 80)
print("PIPELINE COMPLETE!")
print("=" * 80)
print(f"✓ Processed {n_samples} compounds")
print(f"✓ Extracted {X.shape[1]} features per spectrum")
print(f"✓ Classified {y.shape[1]} functional groups")
print(f"✓ Best model: {best_model_name} (F1-Micro: {comparison_df.loc[best_model_name, 'F1-Micro']:.4f})")
print("=" * 80)